# Measles vaccine example

This notebook reproduces a [visualization by the Wall Street Journal](http://graphics.wsj.com/infectious-diseases-and-vaccines/#b02g20t20w15) about the incidence of measles over time, which the brilliant [Brian Granger](https://github.com/ellisonbg) adapted into an [example for the Altair library](http://nbviewer.jupyter.org/github/ellisonbg/altair/blob/master/altair/notebooks/12-Measles.ipynb). Here we will try to reproduce the plot using HoloViews. We begin by loading the data:

In [ ]:
import numpy as np
import holoviews as hv
import pandas as pd
hv.notebook_extension('matplotlib')
%output backend='matplotlib' fig='svg'

## Declaring data

In [ ]:
url = 'https://raw.githubusercontent.com/blmoore/blogR/master/data/measles_incidence.csv'
data = pd.read_csv(url, skiprows=2, na_values='-')

yearly_data = data.drop('WEEK', axis=1).groupby('YEAR').sum().reset_index()
measles = pd.melt(yearly_data, id_vars=['YEAR'], var_name='State', value_name='Incidence')

heatmap = hv.HeatMap(measles, label='Measles Incidence')
aggregate = hv.Dataset(heatmap).aggregate('YEAR', np.mean, np.std)

vline = hv.VLine(1963)
marker = hv.Text(1964, 800, 'Vaccine introduction', halign='left')

agg = hv.ErrorBars(aggregate) * hv.Curve(aggregate)

## Matplotlib

In [ ]:
hm_opts = dict(aspect=1.4, invert_yaxis=True, show_values=False, show_frame=False,
               xrotation=90, labelled=[], toolbar='above', xaxis=None)
overlay_opts = dict(aspect=3, show_title=False, bgcolor='white', show_legend=False, show_frame=False)
vline_opts = dict(color='black')
layout_opts = dict(aspect_weight=1, fig_size=300, vspace=0, sublabel_format=None)

opts = {'Layout': {'plot': layout_opts}, 'HeatMap': {'plot': hm_opts},
        'Overlay': {'plot': overlay_opts}, 'VLine': {'style': vline_opts}}

(heatmap + agg * vline * marker)(opts).cols(1)

## Bokeh

In [ ]:
%%output backend='bokeh'
hm_opts = dict(width=900, height=500, tools=['hover'], logz=True, invert_yaxis=True,
            xrotation=90, labelled=[], toolbar='above', xaxis=None)
overlay_opts = dict(width=900, height=200, show_title=False)
vline_opts = dict(line_color='black')

opts = {'HeatMap': {'plot': hm_opts}, 'Overlay': {'plot': overlay_opts}, 'VLine': {'style': vline_opts}}

(heatmap + agg * vline * marker)(opts).cols(1)